# MGE detection

Uses the phyletic distribution pattern based mobility score to detect MGEs in genomes.

## Preparation

In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
score_threshold = 0.3
n_threshold=10
fraction_threshold=0.1
n_scc_threshold=5

In [10]:
project_path = Path().resolve().parent
input_path = project_path / "results" / "mobility_files" /  "thesis"
output_path = project_path / "results" / "MGE_files"
#os.mkdir(output_path)

In [11]:
genome_contig="GCA_000006865.1-AE005176.1"
genome_contig_file=str(genome_contig+".csv")
genome2="GCA_001868825.1-MLNO01000011.1"
genome2_file=str(genome2+".csv")

## Load mobility frame

In [12]:
frame=pd.read_csv(input_path / genome2_file, index_col=0)

In [13]:
frame

,order,orthogroup,gene,accessory_fraction,tree_score,count,accessory
0,1,F33765_1,MLNO01000011.1_1,0.000000,NaN,2,False
1,2,F01444_3,MLNO01000011.1_2,0.014493,NaN,1,False
2,3,F02943_1,MLNO01000011.1_3,0.260870,NaN,4,False
3,4,F00978_1,MLNO01000011.1_4,0.500000,NaN,2,False
4,5,F02552_1,MLNO01000011.1_5,0.730769,NaN,2,False
5,6,F19307_1,MLNO01000011.1_6,0.523810,NaN,2,False
6,7,F00386_1,MLNO01000011.1_7,0.694444,NaN,1,False
7,8,F15837_1,MLNO01000011.1_8,0.666667,NaN,1,False
8,9,F09060_1,MLNO01000011.1_9,0.622222,NaN,2,False
9,10,F03841_1,MLNO01000011.1_10,0.234375,NaN,3,False


## Sophisticated MGE detection (front and back boundary detection)

In [7]:
###
i=0
exc=0
start_genes=[]
end_genes=[]
indices=[]
lengths=[]
for index,row in frame[::-1].iterrows():
    consecutive=(row.accessory_fraction>score_threshold)
    if consecutive:
        i=i+1
        #print(i)
        #print(index)
    else:
        exc=exc+1
        if(row.accessory|(row['count']>1)|(exc<fraction_threshold*i)):
            i=i+1
            #print(index)
        else:
            if i>n_threshold:
                indices.append(index)
                lengths.append(i)
                #print(index)
                #start_genes.append(index+1-i)
                #end_genes.append(index+1)
                print("Possible MGE: " + str(index+2) + "-" + str(index+2+i))
            i=0
            exc=0
                
# find consecutive genes with mobility score exceeding a threshold 

Possible MGE: 1696-1718
Possible MGE: 1541-1555
Possible MGE: 1146-1203
Possible MGE: 946-964
Possible MGE: 858-875
Possible MGE: 841-852
Possible MGE: 736-804
Possible MGE: 500-554
Possible MGE: 410-440
Possible MGE: 102-119


In [21]:
def end_detection(mob_frame):
    genes,lengths=get_indices_lengths(mob_frame[::-1])
    end_genes=list(np.array(genes)+np.array(lengths))
    return [gene+1 for gene in end_genes][::-1], lengths[::-1]

In [22]:
def start_detection(mob_frame):
    genes,lengths=get_indices_lengths(mob_frame)
    start_genes=list(np.array(genes)-np.array(lengths))
    return [gene+1 for gene in start_genes], lengths

In [23]:
def get_indices_lengths(frame, score_threshold=score_threshold, fraction_threshold=fraction_threshold, n_threshold=n_threshold, n_scc_threshold=n_scc_threshold):
    i=0
    exc=0
    n_scc_consecutive=0
    indices=[]
    lengths=[]
    for index,row in frame.iterrows():
        #consecutive=(row.accessory_fraction>score_threshold)
        if row.accessory_fraction>score_threshold:
            # look for consecutive genes with an elevated mobility score
            i=i+1
        elif row.accessory|(row['count']>1):
            #exc are exceptions, so genes that do not have an elevated mobility score
            exc=exc+1
            # in case these exceptions are multi-copy or accessory, or when they do not appear too close to the start of the potential MGE, they are tolerated
            i=i+1
        elif ((exc<fraction_threshold*i)&(n_scc_consecutive<n_scc_threshold)):
            exc=exc+1
            # in case these exceptions do not appear too often and not too often in a consecutive fashion, they are also tolerated
            i=i+1
            n_scc_consecutive = n_scc_consecutive + 1
        elif i>n_threshold:
            # if the number of consecutive genes that belong to a potential MGE is large enough, the potential MGE is predicted to be a true MGE
            # in this case, the index and length of the MGE are saved
            indices.append(index)
            lengths.append(i)
            # parameters are set to zero again, to find a new (potential) MGE
            i=0
            exc=0
            n_scc_consecutive = 0
        else:
            # this case captures everything else: genes that are not part of an MGEs
            # parameters are set to zero again, to find a new (potential) MGE
            i=0
            exc=0
            n_scc_consecutive = 0
    return indices,lengths
                

In [131]:
def generate_output(genome_contig_file):
    """Generates csv files, containing a columns with the contig, a column with a number indicating an individual MGE, and a columns with gene_nrs, indicating the genes that belong to a particular MGE."""
    contig_extension=genome_contig_file.split('-')[1]
    contig=str(contig_extension.split('.')[0]+'.'+contig_extension.split('.')[1])
    MGE_list=[]
    mob_frame=pd.read_csv(input_path / genome_contig_file, index_col=0)
    start_genes, start_lengths=start_detection(mob_frame)
    end_genes, end_lengths=end_detection(mob_frame)
    MGE=0
    # find matching elements in end_genes and start_genes (some potential MGEs will only be detected in one direction)
    if (len(start_genes)>0)&(len(end_genes)>0):
        for i in range(0,len(start_genes)):
            for j in range(0, len(end_genes)):
                if (start_genes[i]+start_lengths[i] >= end_genes[j])&(end_genes[j]-end_lengths[j] <= start_genes[i])&(start_genes[i]<end_genes[j]):
                    #print("length: "+str(start_lengths[i]))
                    #print("start: "+str(start_genes[i]))
                    #print("end: "+str(end_genes[j]))
                    MGE=MGE+1
                    for gene in range(start_genes[i], end_genes[j]+1):
                        MGE_list.append({'contig':contig,'MGE':MGE,'gene_nr':gene})
                    
                
    MGE_frame = pd.DataFrame.from_records(MGE_list)
    return MGE_frame 

In [132]:
generate_output(genome_contig_file)

,contig,MGE,gene_nr
0,AE005176.1,1,23
1,AE005176.1,1,24
2,AE005176.1,1,25
3,AE005176.1,1,26
4,AE005176.1,1,27
...,...,...,...
425,AE005176.1,16,2050
426,AE005176.1,16,2051
427,AE005176.1,16,2052
428,AE005176.1,16,2053


In [133]:
generate_output(genome2_file)[0:50]

,contig,MGE,gene_nr
0,AE004092.2,1,102
1,AE004092.2,1,103
2,AE004092.2,1,104
3,AE004092.2,1,105
4,AE004092.2,1,106
5,AE004092.2,1,107
6,AE004092.2,1,108
7,AE004092.2,1,109
8,AE004092.2,1,110
9,AE004092.2,1,111


## Sophisticated MGE detection (MGE joining)

In [8]:
# parameters
score_threshold=0.3
initial_n_threshold=5
n_threshold=10
distance_threshold=5
fraction_threshold=0.1

In [19]:
def detect_mges_simple(frame, n_threshold=initial_n_threshold):
    i=0
    start_genes=[]
    end_genes=[]
    for index,row in frame.iterrows():
        consecutive=row.accessory|(row['count']>1)
        if consecutive:
            i=i+1
        else:
            if i>n_threshold:
                start_genes.append(index+1-i)
                end_genes.append(index+1)
                i=0
    return start_genes, end_genes

NameError: name 'initial_n_threshold' is not defined

In [80]:
def join_initial_mges(start_genes, end_genes, distance_threshold=distance_threshold, n_threshold=n_threshold):
    
    #print(len(start_genes))
    #print(len(end_genes))
    
    indices_to_join=[]
    for index in range(len(start_genes)):
        if index != 0:
            if(start_genes[index]<end_genes[index-1]+distance_threshold):
                indices_to_join.append(index)
    
    for index in range(len(indices_to_join))[::-1]:
        start_genes.remove(start_genes[index])
        end_genes[index]=end_genes[index-1]
        end_genes.remove(end_genes[index-1])
    #print(len(start_genes))
    #print(len(end_genes))
        
    for index in range(len(start_genes))[::-1]:
        if end_genes[index]-start_genes[index]<n_threshold:
            start_genes.remove(start_genes[index])
            end_genes.remove(end_genes[index])
            #print("too small")
    #print(len(start_genes))
    #print(len(end_genes))
    
    return start_genes, end_genes

In [87]:
def generate_output_joining(genome_contig_file):
    """Generates csv files, containing a columns with the contig, a column with a number indicating an individual MGE, and a columns with gene_nrs, indicating the genes that belong to a particular MGE."""
    contig_extension=genome_contig_file.split('-')[1]
    contig=str(contig_extension.split('.')[0]+'.'+contig_extension.split('.')[1])
    MGE_list=[]
    mob_frame=pd.read_csv(input_path / genome_contig_file, index_col=0)
    start_genes, end_genes=detect_mges_simple(frame)
    start_genes, end_genes=join_initial_mges(start_genes, end_genes)
    MGE=1
    for index in range(len(start_genes)):
        for gene in range(start_genes[index], end_genes[index]+1):
            MGE_list.append({'contig':contig,'MGE':MGE,'gene_nr':gene})
        MGE=MGE+1
    MGE_frame = pd.DataFrame.from_records(MGE_list)
    return MGE_frame 

In [75]:
start_genes, end_genes=detect_mges_simple(frame)

In [76]:
start_genes, end_genes=join_initial_mges(start_genes, end_genes)

11
11


In [77]:
start_genes

[173, 425, 502, 740, 827, 858, 1047, 1147, 1543, 1633, 1695]

In [78]:
end_genes

[183, 438, 554, 804, 837, 875, 1058, 1203, 1554, 1643, 1718]

In [88]:
MGE_frame=generate_output_joining(genome_contig_file)

In [89]:
MGE_frame

,contig,MGE,gene_nr
0,AE005176.1,1,173
1,AE005176.1,1,174
2,AE005176.1,1,175
3,AE005176.1,1,176
4,AE005176.1,1,177
...,...,...,...
283,AE005176.1,11,1714
284,AE005176.1,11,1715
285,AE005176.1,11,1716
286,AE005176.1,11,1717


## Simple MGE detection

In [38]:
def detect_mges_simple(frame, n_threshold=n_threshold):
    i=0
    start_genes=[]
    end_genes=[]
    for index,row in frame.iterrows():
        consecutive=row.accessory|(row['count']>1)
        if consecutive:
            i=i+1
            pos = index+1
            print(str(pos) +" is multi_copy/accessory")
        else:
            if i>=n_threshold:
                print(str(pos) + ": element saved")
                start_genes.append(index+1-i)
                end_genes.append(index+1)
                i=0
            else:
                i=0
    print(start_genes)
    print(end_genes)
    return start_genes, end_genes


In [39]:
def generate_output_simple(genome_contig_file):
    """Generates csv files, containing a columns with the contig, a column with a number indicating an individual MGE, and a columns with gene_nrs, indicating the genes that belong to a particular MGE."""
    contig_extension=genome_contig_file.split('-')[1]
    contig=str(contig_extension.split('.')[0]+'.'+contig_extension.split('.')[1])
    MGE_list=[]
    mob_frame=pd.read_csv(input_path / genome_contig_file, index_col=0)
    start_genes, end_genes=detect_mges_simple(mob_frame)
    for i in range(0,len(start_genes)):
        for gene in range(start_genes[i], end_genes[i]+1):
            MGE_list.append({'contig':contig,'MGE':i+1,'gene_nr':gene})
    MGE_frame = pd.DataFrame.from_records(MGE_list)
    return MGE_frame 

In [40]:
generate_output_simple(genome2_file)

1 is multi_copy/accessory
3 is multi_copy/accessory
4 is multi_copy/accessory
5 is multi_copy/accessory
6 is multi_copy/accessory
9 is multi_copy/accessory
10 is multi_copy/accessory
11 is multi_copy/accessory
15 is multi_copy/accessory
18 is multi_copy/accessory
20 is multi_copy/accessory
22 is multi_copy/accessory
23 is multi_copy/accessory
[]
[]


""


In [137]:
generate_output_simple(genome2_file)[0:50]

,contig,MGE,gene_nr
0,AE004092.2,1,92
1,AE004092.2,1,93
2,AE004092.2,1,94
3,AE004092.2,1,95
4,AE004092.2,1,96
5,AE004092.2,1,97
6,AE004092.2,1,98
7,AE004092.2,1,99
8,AE004092.2,1,100
9,AE004092.2,1,101
